In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch_geometric.utils import dense_to_sparse, add_self_loops, get_laplacian

In [60]:
Traffic = pd.read_csv("data/traffic.csv")
Adjacency = np.load('data/Adjacency.npy')

In [61]:
# Convert adjacency matrix to PyTorch tensor and normalize it
adj_matrix = torch.tensor(Adjacency, dtype=torch.float)
adj_matrix += torch.eye(adj_matrix.size(0))
degree_matrix = adj_matrix.sum(dim=1)
d_inv_sqrt = torch.diag(degree_matrix.pow(-0.5))
adj_matrix_norm = d_inv_sqrt @ adj_matrix @ d_inv_sqrt
edge_index, edge_weight = dense_to_sparse(adj_matrix_norm)

In [63]:
time_steps = Traffic['timestep'].nunique()
num_locations = Traffic['location'].nunique()

all_features = torch.tensor(Traffic[['occupy', 'speed']].values, dtype=torch.float)
feature_mean = all_features.mean(dim=0)
feature_std = all_features.std(dim=0)

all_labels = torch.tensor(Traffic['flow'].values, dtype=torch.float)
label_mean = all_labels.mean()
label_std = all_labels.std()

node_features_per_time = []
labels_per_time = []

In [64]:
for timestep in range(1, time_steps + 1):
    timestep_data = Traffic[Traffic['timestep'] == timestep]
    features = torch.tensor(timestep_data[['occupy', 'speed']].values, dtype=torch.float)
    labels = torch.tensor(timestep_data['flow'].values, dtype=torch.float)
    
    features = (features - feature_mean) / feature_std
    labels = (labels - label_mean) / label_std

    node_features_per_time.append(features)
    labels_per_time.append(labels)

In [65]:
# Sliding window (12 steps)
window_size = 12
node_features_sliding = []
labels_sliding = []

for i in range(time_steps - window_size):
    node_features_sliding.append(torch.stack(node_features_per_time[i:i + window_size]))
    labels_sliding.append(labels_per_time[i + window_size])

node_features_sliding = torch.stack(node_features_sliding)  # Shape: (num_samples, window_size, num_nodes, num_features)
labels_sliding = torch.stack(labels_sliding)  # Shape: (num_samples, num_nodes)

In [69]:
# Model definition
class SpatioTemporalGNN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(SpatioTemporalGNN, self).__init__()
        self.gcn = GCNConv(in_channels, hidden_channels)
        self.lstm = nn.LSTM(hidden_channels, hidden_channels, batch_first=True)
        self.fc = nn.Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index, edge_weight):
        batch_size, time_steps, num_nodes, in_channels = x.size()
        outputs = []

        edge_index_batch, edge_weight_batch = self._batch_edge_index(edge_index, edge_weight, num_nodes, batch_size)

        for t in range(time_steps):
            x_t = x[:, t, :, :].reshape(batch_size * num_nodes, in_channels)
            x_t = torch.relu(self.gcn(x_t, edge_index_batch, edge_weight_batch))
            x_t = x_t.view(batch_size, num_nodes, -1)
            outputs.append(x_t)

        x = torch.stack(outputs, dim=1)
        x = x.view(batch_size * num_nodes, time_steps, -1)
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = x.view(batch_size, num_nodes, -1)
        x = self.fc(x)
        return x

    def _batch_edge_index(self, edge_index, edge_weight, num_nodes, batch_size):
        edge_indices = []
        edge_weights = []
        for i in range(batch_size):
            offset = i * num_nodes
            edge_indices.append(edge_index + offset)
            edge_weights.append(edge_weight)
        edge_index_batch = torch.cat(edge_indices, dim=1)
        edge_weight_batch = torch.cat(edge_weights)
        return edge_index_batch, edge_weight_batch

In [71]:
# Model initialization
in_channels = node_features_sliding.shape[3]
hidden_channels = 16
out_channels = 1
model = SpatioTemporalGNN(in_channels, hidden_channels, out_channels)

loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
epochs = 100

# Training
losses = []

In [ ]:
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(node_features_sliding, edge_index, edge_weight)
    loss = loss_fn(output.squeeze(), labels_sliding)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())
    print(f"Epoch {epoch + 1}, Loss: {loss.item():.4f}")

In [ ]:
# Visualization of Loss
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()